In [ ]:
# 導入本次爬蟲所需的模組
import csv
from datetime import date, datetime, timedelta

import requests
from pyquery import PyQuery as pq

In [ ]:
# 先觀察網頁結構、網址變化

# 用requests及pyquery
res = requests.get("https://ibmi.taiwan-healthcare.org/news_list.php?REFDOCTYPID=0o4dd9ctwhtyumw0&PageNO=1")
doc = pq(res.text)
nextPgDoc = doc

In [ ]:
for Try in range(3):  #或者改成 while 迴圈，直到使用者輸入正確的數字才離開
    try:
        Day = int(input("輸入想要抓取幾天內的新聞_請輸入數字(天)： "))+1
        break
    except:
        print("請輸入一個正確的整數,你還有次{}機會".format(2-Try))
        if 2-Try == 0:
            Day = -2    #為了讓後面的CSV檔可執行寫入，故意輸入一個負數
    
my_News_List = []

pg = 1 
while True:
    res = requests.get("https://ibmi.taiwan-healthcare.org/news_list.php?REFDOCTYPID=0o4dd9ctwhtyumw0&PageNO={}".format(pg))
    doc = pq(res.text)

    doc.make_links_absolute(base_url = res.url) 
#     print(len(doc("table.news-list-table > tr > td"))) #觀察該頁面有無內容
    if datetime.now() - timedelta(days = Day) > datetime.strptime(doc("table.news-list-table > tr:nth-child(2) > td:nth-child(1)").text(),"%Y-%m-%d"):
        break
    for j, k, l in zip(doc("table.news-list-table > tr > td:nth-child(1)").items(), doc("table.news-list-table > tr > td >a").items(), doc("table.news-list-table > tr > td:nth-child(3)").items()):
        #以字典存入對應的資訊
        temp_NewsDict = {}
        newsDate = j.text()
        if datetime.now() - timedelta(days = Day) <= datetime.strptime(newsDate,"%Y-%m-%d"):
            temp_NewsDict["日期"] = j.text()
            temp_NewsDict["標題"] = k.text()
            temp_NewsDict["新聞媒體"] = l.text()
            print(k.text(), k.attr("href")) #印出來檢查有沒有選到新聞標題及連結
            NewsDetail_Doc = pq(k.attr("href"))

            for each_NewsDetail_Doc in NewsDetail_Doc.items():
                temp_NewsDict["新聞內容"] = each_NewsDetail_Doc("div.center-left-value").text()
    #             print(temp_NewsDict) #確認是否有存入對應的資訊到字典
                my_News_List.append(temp_NewsDict)
        else:
            break
    
    pg += 1
    if len(doc("table.news-list-table > tr > td > a")) == 0:
        break

In [ ]:
# 將檔案存成.csv檔
# 在一個 csv 檔案中插入多個字典

labels = ["日期", "標題", "新聞媒體", "新聞內容"]

# 為了防止外部錯誤或關於檔案寫入過程的不相容，使用例外處理
try:
    with open("{}_{}天內_生技醫療新聞.csv".format(datetime.now().strftime("%Y-%m-%d"), Day-1),"w",encoding="UTF-16", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=labels)
        writer.writeheader()
        for elem in my_News_List:
            writer.writerow(elem)
except IOError:
    print("I/O error")